 ## Run in high RAM runtime or will crash OOM error

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Huggingface version

In [0]:
import os
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/transformers/examples')
! pip install -r requirements.txt
! pip install transformers

# os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/transformers/examples/summarization/bart')
# ! wget https://s3.amazonaws.com/datasets.huggingface.co/summarization/cnn_dm.tgz
# ! tar -xzvf cnn_dm.tgz

In [0]:
# To run model
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/transformers/examples/summarization/bart')

! python evaluate_cnn.py \
    /content/drive/My\ Drive/Documents/Imperial/Individual_project/datasets/cnn_dm/test.source \
    /content/drive/My\ Drive/Documents/Imperial/Individual_project/datasets/cnn_dm/cnn_test_summaries.txt \
    bart-large-cnn

In [16]:
import os
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/transformers/examples/summarization/bart')
! ls cnn_dm/

test.source  test.target  train.source	train.target  val.source  val.target


In [26]:
# ROUGE eval
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/rouge/')
! python runner.py \
    /content/drive/My\ Drive/Documents/Imperial/Individual_project/datasets/cnn_dm/test.target \
    /content/drive/My\ Drive/Documents/Imperial/Individual_project/datasets/cnn_dm/huggingface_sums/cnn_test_summaries.txt

-----   Avg. i=0  -----
-----  -----  -----
0.311  0.118  0.303
-----  -----  -----
-----   Avg. i=1000  -----
-----  -----  -----
0.421  0.193  0.416
-----  -----  -----
-----   Avg. i=2000  -----
-----  -----  -----
0.422  0.193  0.418
-----  -----  -----
-----   Avg. i=3000  -----
-----  -----  -----
0.422  0.191  0.414
-----  -----  -----
-----   Avg. i=4000  -----
-----  -----  -----
0.406  0.181  0.396
-----  -----  -----
-----   Avg. i=5000  -----
-----  -----  -----
0.403  0.183  0.390
-----  -----  -----
-----   Avg. i=6000  -----
-----  -----  -----
0.405  0.186  0.395
-----  -----  -----
-----   Avg. i=7000  -----
-----  -----  -----
0.398  0.178  0.386
-----  -----  -----
-----   Avg. i=8000  -----
-----  -----  -----
0.402  0.181  0.390
-----  -----  -----
-----   Avg. i=9000  -----
-----  -----  -----
0.403  0.185  0.391
-----  -----  -----
-----   Avg. i=10000  -----
-----  -----  -----
0.403  0.186  0.390
-----  -----  -----
-----   Avg. i=11000  -----
-----  -----  ---

## Fairseq version

In [0]:
import os
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/datasets/cnn_dm')
! pip install fairseq
! ls

In [22]:
# To run model
import torch
from fairseq.models.bart import BARTModel

bart = torch.hub.load('pytorch/fairseq', 'bart.large.cnn')
bart.cuda()
bart.eval()
bart.half()
count = 1
bsz = 32
with open('test.source') as source, open('fairseq_sums/test.hypo', 'w') as fout:
    sline = source.readline().strip()
    slines = [sline]
    for sline in source:
        if count % bsz == 0:
            with torch.no_grad():
                hypotheses_batch = bart.sample(slines, beam=4, lenpen=2.0, max_len_b=140, min_len=55, no_repeat_ngram_size=3)

            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()
            slines = []

        slines.append(sline.strip())
        count += 1
    if slines != []:
        hypotheses_batch = bart.sample(slines, beam=4, lenpen=2.0, max_len_b=140, min_len=55, no_repeat_ngram_size=3)
        for hypothesis in hypotheses_batch:
            fout.write(hypothesis + '\n')
            fout.flush()

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip
100%|██████████| 3715581256/3715581256 [02:44<00:00, 22616247.62B/s]


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/bart.large.cnn.tar.gz from cache at /root/.cache/torch/pytorch_fairseq/6ee6af21df382be253b815b53834cd71f61b7ebb83deaa2d10c5d3f94db833cb.3a612b08cd8ff157daf01330f2160f0889831fcb543ec25aa63e38c726167380
extracting archive file /root/.cache/torch/pytorch_fairseq/6ee6af21df382be253b815b53834cd71f61b7ebb83deaa2d10c5d3f94db833cb.3a612b08cd8ff157daf01330f2160f0889831fcb543ec25aa63e38c726167380 to temp dir /tmp/tmpopvubi0d
| [source] dictionary: 50264 types
| [target] dictionary: 50264 types


1042301B [00:00, 1835757.19B/s]
456318B [00:00, 1208011.78B/s]
/pytorch/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [24]:
# ROUGE eval
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/rouge/')
! python runner.py \
    /content/drive/My\ Drive/Documents/Imperial/Individual_project/datasets/cnn_dm/test.target \
    /content/drive/My\ Drive/Documents/Imperial/Individual_project/datasets/cnn_dm/fairseq_sums/test.hypo

-----   Avg. i=0  -----
-----  -----  -----
0.308  0.117  0.300
-----  -----  -----
-----   Avg. i=1000  -----
-----  -----  -----
0.419  0.192  0.414
-----  -----  -----
-----   Avg. i=2000  -----
-----  -----  -----
0.422  0.193  0.417
-----  -----  -----
-----   Avg. i=3000  -----
-----  -----  -----
0.420  0.190  0.412
-----  -----  -----
-----   Avg. i=4000  -----
-----  -----  -----
0.404  0.180  0.394
-----  -----  -----
-----   Avg. i=5000  -----
-----  -----  -----
0.403  0.183  0.390
-----  -----  -----
-----   Avg. i=6000  -----
-----  -----  -----
0.405  0.185  0.394
-----  -----  -----
-----   Avg. i=7000  -----
-----  -----  -----
0.397  0.177  0.385
-----  -----  -----
-----   Avg. i=8000  -----
-----  -----  -----
0.403  0.182  0.390
-----  -----  -----
-----   Avg. i=9000  -----
-----  -----  -----
0.403  0.186  0.391
-----  -----  -----
-----   Avg. i=10000  -----
-----  -----  -----
0.404  0.187  0.390
-----  -----  -----
-----   Avg. i=11000  -----
-----  -----  ---

In [35]:
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/files2rouge/')

! python -m files2rouge test.hypo.tokenized test.hypo.target

Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.6/runpy.py", line 142, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.6/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/content/drive/My Drive/Documents/Imperial/Individual_project/files2rouge/files2rouge/__init__.py", line 2, in <module>
    from files2rouge.files2rouge import run
  File "/content/drive/My Drive/Documents/Imperial/Individual_project/files2rouge/files2rouge/files2rouge.py", line 20, in <module>
    import pyrouge
ModuleNotFoundError: No module named 'pyrouge'
